In [28]:
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch

In [29]:
csv_path = r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_train.csv"
data = pd.read_csv(csv_path)

In [30]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
label = data.iloc[:, 0]
type(label)
label.values
label = label.values

In [32]:
label.shape

(60000,)

In [33]:
train_set = data.iloc[:, 1:].values
train_set

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [34]:
train_set.shape

(60000, 784)

In [35]:
train_set = train_set.astype(np.uint8)
train_set

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [36]:
train_set = train_set.reshape(60000, 28, 28)
train_set.shape

(60000, 28, 28)

In [37]:
train_set[0].shape

(28, 28)

In [38]:
label

array([2, 9, 6, ..., 8, 8, 7], dtype=int64)

In [39]:
target = int(label[0])
target

2

In [40]:
transform = transforms.Compose([transforms.ToTensor()])

In [41]:
img = transform(train_set)
img.shape
#有问题

torch.Size([28, 60000, 28])

In [42]:
len(train_set)

60000

In [55]:
class Fasion_Mnist(Dataset):
    def __init__(self, csv_path, transform=None):
        data = pd.read_csv(csv_path)       
        label = data.iloc[:, 0].values
        train_set = data.iloc[:, 1:].values.astype(np.uint8).reshape(len(label), 28, 28)
        # train_set /= 255.0 
       
        self.train_set = train_set
        self.train_label = label
        self.transform = transform
    
    def __getitem__(self, index):
        img = self.train_set[index]
        target = int(self.train_label[index])
        #为什么要转换成int呢？
        if self.transform is not None:
            img = self.transform(img)
        return img, target
    def __len__(self):
        return len(self.train_set)

train_dataset = Fasion_Mnist(r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_train.csv", transform=transform)
test_dataset = Fasion_Mnist(r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_test.csv", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [56]:
train_dataset

In [57]:
train_loader

In [ ]:
for batch_idx, data in enumerate(train_loader, 0):
    inputs, target = data
inputs

In [60]:
inputs.shape, target.shape
#为啥是32呢？

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [48]:
target

tensor([1, 3, 8, 9, 4, 9, 4, 0, 6, 2, 1, 2, 8, 3, 5, 9, 8, 0, 7, 4, 5, 5, 1, 9,
        6, 7, 4, 0, 5, 1, 8, 3, 9, 5, 7, 3, 3, 1, 5, 1, 9, 8, 2, 7, 4, 5, 0, 8,
        0, 5, 1, 2, 0, 0, 4, 3, 8, 0, 8, 9, 7, 3, 6, 6, 0, 8, 7, 6, 3, 2, 7, 7,
        0, 3, 4, 2, 6, 1, 3, 1, 9, 8, 4, 7, 1, 3, 9, 5, 8, 7, 8, 3, 7, 3, 7, 8])

In [49]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 6, kernel_size=3),
            torch.nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(6, 16, kernel_size=3),
            torch.nn.MaxPool2d(2,2),
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(400, 120),
            torch.nn.Linear(120, 84),
            torch.nn.Linear(84, 10)
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv1(x)  
        x = self.conv2(x)  
        x = x.view(batch_size, -1) 
        x = self.fc(x)
        return x  

model = Net()

In [50]:
outputs = model(inputs)
outputs.shape

torch.Size([96, 10])

In [51]:
outputs.data[4]
#detach()方法

tensor([-0.0682,  0.0360,  0.0291,  0.0039, -0.1158,  0.0303,  0.0786,  0.0080,
         0.0019,  0.0330])

In [52]:
_, predicted = torch.max(outputs.data, dim=1)
predicted
#有点问题

tensor([6, 6, 1, 6, 6, 1, 6, 6, 6, 6, 6, 6, 1, 1, 6, 2, 6, 6, 6, 6, 6, 1, 6, 6,
        6, 1, 6, 6, 6, 6, 1, 6, 6, 1, 6, 6, 6, 6, 6, 6, 1, 6, 6, 1, 6, 6, 6, 1,
        9, 9, 6, 6, 6, 6, 6, 1, 1, 6, 1, 6, 6, 6, 6, 6, 6, 1, 6, 6, 6, 6, 6, 1,
        6, 6, 6, 9, 6, 6, 6, 6, 1, 1, 6, 6, 6, 6, 6, 6, 1, 6, 6, 6, 6, 6, 1, 6])

In [53]:
inputs.shape[0]

96